This notebook performs maximum likelihood estimation (MLE) for one participant's choices, under a feature reinforcement learning with decay model. The fitting procedure tests best fit (MLE) parameters on choice data from a left-out game.

In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from scipy.stats import dirichlet
from scipy.optimize import minimize
import time as time
import pickle 

# Custom dependencies
import sys
import os
sys.path.append(os.getcwd() + '/libraries/')
from World import World
from FeatureRL import Agent as FRL
from FeatureRL import train_frl_choice
from Data import Data, extract_vars 
from fitting import Fit 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Instantiate world

In [5]:
## Set world properties.
n_dims = 3
n_feats_per_dim = 3
world_h = 0
world_p_high = 0.75
world_p_low = 0.25
outcome = 1
 
world = World(n_dims, n_feats_per_dim, world_h, world_p_high, world_p_low, outcome)

## 2. Load data

In [6]:
## Set paths.
behav_path = os.getcwd() + '/data/subj1_behav.csv'
et_path = os.getcwd() + '/data/subj1_gaze.csv'
fit_path = os.getcwd() + '/fits/subj1_MLE_fit.csv'
fit_file = open(fit_path, 'wb')

## Load datafiles.
# Behavior. 
behav_data_subj = pd.read_csv(behav_path)
# Feature level attention.
et_data_subj = pd.read_csv(et_path, names = np.arange(9)+1)
et_data_subj['Trial'] = np.arange(len(behav_data_subj['Trial'].values))+1
et_data_subj.reset_index(inplace = True, drop = True)

## Make data object.
data = Data(behav_data_subj, et_data_subj)

## 3. Run MLE with leave-one-game out cross-validation 

In [9]:
start_time = time.time()

## Set number of iterations.
n_iter = 5

## Set bounds.
bnds = [(1e-6,1), (1e-6,1), (1e-6,500)]

## Initialize output arrays.
results = np.zeros((n_iter,data.n_games,len(bnds)+1))
initial_conditions = np.zeros((n_iter,len(bnds)))

## Run optimization with different initial conditions.
for i in np.arange(n_iter): 
    
    print('iteration #', i+1)
    
    ## Set initial conditions.
    init_cond = [np.random.uniform(0, 1), np.random.uniform(0, 1), np.random.uniform(2, 5)]
    initial_conditions[i,:] = init_cond
    print('initial conditions:', init_cond)
        
    for g in np.arange(data.n_games):
    
        ## Set current test game. 
        test_game = g+1
        print('predicting game:', test_game)

        ## Split behavior into training and test data.
        behav_training_data, behav_test_data, et_training_data, et_test_data = data.split_data(test_game)

        ## Train model.
        res = minimize(train_frl_choice, init_cond, args=(behav_training_data, et_training_data), method='L-BFGS-B', bounds=bnds, options={'maxfun': 150, 'maxiter': 100})
        results[i,test_game-1,0:len(bnds)] = res.x

        ## Evaluate model on test set.
        test_trials = behav_test_data.Trial.unique()        
        extracted_data = extract_vars(behav_test_data, et_test_data, test_trials)

        ## Set parameters.
        # Default values set to 0.
        test_params = {'learning_rate': res.x[0],
                    'decay_rate': res.x[1],
                    'beta_value_choice': res.x[2],
                    'beta_value_gaze': 0,
                    'beta_center_dim': 0,
                    'beta_center_feat': 0,
                    'w_init': 0,
                    'decay_target': 0,
                    'precision': 0}
        del res

        ## Instantiate and run MLE agent.
        frl_mle = FRL(world, test_params)
        W, test_lik = frl_mle.choice_likelihood(world, extracted_data)
        results[i,test_game-1,-1] = test_lik

        n_trials = np.shape(extracted_data['outcomes'])
        
        print('maximum likelihood learning rate:', test_params['learning_rate'])
        print('maximum likelihood decay rate:', test_params['decay_rate'])
        print('maximum likelihood inverse temperature:', test_params['beta_value_choice'])
        print('test set log likelihood:', test_lik)
        
        print('test set likelihood per trial:', np.exp(test_lik/n_trials))
        
elapsed_time = time.time() - start_time
print('elapsed time: ' + str((np.round(elapsed_time,decimals=3))) + ' seconds')

iteration # 1
initial conditions: [0.5160651250504203, 0.23513764593355557, 2.437976292213742]
predicting game: 1
maximum likelihood learning rate: 0.1809820684351274
maximum likelihood decay rate: 0.4172528369932521
maximum likelihood inverse temperature: 8.053703369239278
test set log likelihood: -15.585405281077225
test set likelihood per trial: [0.44030637]
predicting game: 2
maximum likelihood learning rate: 0.17470801566047384
maximum likelihood decay rate: 0.4012236300295424
maximum likelihood inverse temperature: 7.857744398405978
test set log likelihood: -8.154291184782826
test set likelihood per trial: [0.65104714]
predicting game: 3
maximum likelihood learning rate: 0.183909497872302
maximum likelihood decay rate: 0.4540917612833063
maximum likelihood inverse temperature: 7.726652007708379
test set log likelihood: -10.48287240602668
test set likelihood per trial: [0.59206218]
predicting game: 4
maximum likelihood learning rate: 0.1728568381078057
maximum likelihood decay rat

KeyboardInterrupt: 

In [ ]:
## Initialize and save fit object.
fit = Fit(subj_id, n_iter, initial_conditions, results, elapsed_time)
pickle.dump(fit, fit_file)
fit_file.close()

In [ ]:
## Test that fit was saved correctly. 
new_fit_file = open(fit_path,'rb')
loaded_fit = pickle.load(new_fit_file)

In [ ]:
loaded_fit.elapsed_time